# Evaluate ResNet50

fully-connected architecture modified to 1024, 512, 256 instead of 2048, 1024

Import relevant libraries

In [25]:
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

import pandas as pd
import tensorflow as tf
import wandb

Initialise Weights and Biases API

In [ ]:
wandb.login()
wandb.init(name='resnet50_v2', project='cz4042')
display

Set seed to reproduce results of random operations

In [2]:
seed = 0

Prevent GPU memory usage from exceeding GPU memory

In [3]:
# reference: https://github.com/tensorflow/tensorflow/issues/25160#issuecomment-643703167
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 4)],
        )
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), 'Physical GPUs,', len(logical_gpus), 'Logical GPUs')
    except RuntimeError as error:
        # Virtual devices must be set before GPUs have been initialized
        print(error)
else:
    print('No GPU detected!')

1 Physical GPUs, 1 Logical GPUs


Load data

In [6]:
data_file_path = '/home/burntice/0_repositories/CZ4042/project/data/aligned_gender.csv'
data_dir_path = '/home/burntice/3_data/Adience/'

dataframe = pd.read_csv(data_file_path, sep='\t', names=['datadir', 'gender'])

In [7]:
# confirm dataframe is ok
dataframe.head()

,datadir,gender
0,datadir gender,NaN
1,aligned/30601258@N03/landmark_aligned_face.2.1...,1.0
2,aligned/30601258@N03/landmark_aligned_face.3.1...,1.0
3,aligned/30601258@N03/landmark_aligned_face.2.1...,1.0
4,aligned/30601258@N03/landmark_aligned_face.4.1...,0.0


In [8]:
# check number of data in dataframe
dataframe.shape

(12195, 2)

In [10]:
# check percentage of gender in data
dataframe['gender'].value_counts() / dataframe.shape[0] * 100

0.0    52.660927
1.0    47.330873
Name: gender, dtype: float64

Not very imbalanced; good.

In [11]:
x_train, x_test, y_train, y_test = train_test_split(
    dataframe['datadir'],
    dataframe['gender'],
    test_size=0.2,
    random_state=seed,
)

In [13]:
# checking to make sure everything is alright
x_train.head()

6753     aligned/7636528@N03/landmark_aligned_face.1052...
8876     aligned/113650443@N02/landmark_aligned_face.14...
9269     aligned/48647239@N03/landmark_aligned_face.151...
1022     aligned/37303189@N08/landmark_aligned_face.84....
10913    aligned/10280355@N07/landmark_aligned_face.189...
Name: datadir, dtype: object

In [22]:
train_df = pd.concat([x_train, y_train], axis=1)
test_df = pd.concat([x_test, y_test], axis=1)

In [23]:
train_df.shape

(9756, 2)

In [24]:
train_df['gender'].value_counts() / train_df.shape[0] * 100

0.0    52.97253
1.0    47.01722
Name: gender, dtype: float64

In [9]:
# Hyperparameters
batch_size = 128

# Load images into keras image generator 
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_valid = ImageDataGenerator(rescale=1./255)

# For train generator
train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    directory=data_dir_path,
    x_col='datadir',
    y_col='gender',
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

# For test generator 
val_generator = datagen_valid.flow_from_dataframe(
    dataframe = test_df,
    directory=data_dir_path,
    x_col='datadir',
    y_col='gender',
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

Found 12256 validated image filenames.
Found 4007 validated image filenames.


In [10]:
model = Sequential()

model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='softmax'))
model.layers[0].trainable = False
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 2048)              8192      
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2

In [11]:
# Compile model 
model.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

In [12]:
model.fit(train_generator, epochs=20, validation_data=val_generator)
# model.save_weights('second_model.h5')

Epoch 1/10
  22917/Unknown - 8300s 362ms/step - loss: 6.9277 - accuracy: 0.5457

KeyboardInterrupt: 